In [1]:
using Pkg
Pkg.activate(".")
Pkg.status() 
Pkg.instantiate()
using Revise, Distributions, GLMakie
using TrypColonies

  Activating project at `c:\Users\Andreas\Dropbox\phd\TrypColonies\TrypColonies`


Project TrypColonies v1.0.0-DEV
Status `C:\Users\Andreas\Dropbox\phd\TrypColonies\TrypColonies\Project.toml`
  [336ed68f] CSV v0.10.15
  [35d6a980] ColorSchemes v3.31.0
  [a93c6f00] DataFrames v1.8.0
  [864edb3b] DataStructures v0.19.1
  [31c24e10] Distributions v0.25.122
  [7a1cc6ca] FFTW v1.10.0
  [e9467ef8] GLMakie v0.13.6
  [7073ff75] IJulia v1.31.0
  [6a3955dd] ImageFiltering v0.7.12
⌅ [94395366] ParallelStencil v0.13.6
  [d96e819e] Parameters v0.12.3
  [f517fe37] Polyester v0.7.18
  [37e2e46d] LinearAlgebra v1.11.0
  [9a3f8284] Random v1.11.0
  [9e88b42a] Serialization v1.11.0
  [2f01184e] SparseArrays v1.11.0
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`



(process:16696): GLib-GIO-WARNING **: 17:39:41.851: Failed to open application manifest `C:\Windows\SystemApps\Microsoft.WindowsAppRuntime.CBS_8wekyb3d8bbwe\AppxManifest.xml' for package #66 (`Microsoft.WindowsAppRuntime.CBS_5001.373.1736.100_x64__8wekyb3d8bbwe'): error code 0x2


In [2]:
para_phys = parameters_physical(
    N                       = (1000,1000),
    L                       = (0.015,0.015),
    scale_fac               = 4,
    total_time              = 2000.0,
    walker_speed_phy        = 5*10^-6,
    Diff_coe                = 7*10^-11,
    walker_step_size        = 3,
    adsorption_rate         = 0.05,
    decay_rate              = 0.00001,
    Diameter_colony         = 0.003,
    agent_number            = 250000,
    noise_strength          = 0.3,
    grid_strength           = 150,  
    grid_recover_rate       = 15.0 /2.25,
    radius_collision        = 16,
    radius_tanget           = 15,
    Δt_walker_min           = 0.1,
    )

para = parameters( 
    pa_ph                   = para_phys,
    chemotaxis_flag         = true,
    repulsion_flag          = false,
    repulsion_range         = 2,
    geometry                = "circle",
    start_config_gradient   = "rand",
    )
    
plot_para = plot_parameters(
    arrow_tip_size   = 6,
    arrow_tail_length= 0.00005,
    framerate        = 30, 
    fontsize         = 15,
    refresh_delay    = 0.005,
    res              = (900,1300),
    cor_func_scalar  = order_parameter,
    draw_arrows      = false,
    cor_func_vector  = pair_cor_metric,   
    type_cor_func    = "vector",
    timesteps_corfunc = 100,
    );

In [3]:
para.pa_ph.Δt_diff

0.75

In [4]:
para_phys = parameters_physical(
    N                       = (1000,1000),
    L                       = (0.015,0.015),
    scale_fac               = 4,
    total_time              = 200.0,
    walker_speed_phy        = 5*10^-6,
    Diff_coe                = 7*10^-11,
    walker_step_size        = 3,
    adsorption_rate         = 5.0,
    decay_rate              = 0.01,
    Diameter_colony         = 0.003,
    agent_number            = 2500,
    noise_strength          = 0.8,
    grid_strength           = 2,  
    grid_recover_rate       = 1.0,
    radius_collision        = 12,
    radius_tanget           = 15,
    growth_rate             = 0.2,
    )

para = parameters( 
    pa_ph                   = para_phys,
    chemotaxis_flag         = true,
    repulsion_flag          = false,
    repulsion_range         = 2,
    geometry                = "circle",
    start_config_gradient   = "rand",
    )
    
plot_para = plot_parameters(
    arrow_tip_size   = 6,
    arrow_tail_length= 0.00005,
    framerate        = 30, 
    fontsize         = 15,
    refresh_delay    = 0.005,
    res              = (900,1300),
    cor_func_scalar  = order_parameter,
    draw_arrows      = true,
    cor_func_vector  = pair_cor_metric,   
    type_cor_func    = "vector",
    timesteps_corfunc = 100,
    );

In [5]:
grids = create_grids(para)

grids, Full_agent_list = initialize_system(grids,para)
grids, Agent_list_cicular = make_sys_interactive!(grids,Full_agent_list)
diff_grids = create_diff_grids(para)

makie_config!(plot_para)

f = Figure(size = plot_para.res)
display(f)
ax1,ax2,fig_obs = ini_animation(Agent_list_cicular,para,grids,f,
    plot_para);

pause_button,para_obs = create_buttons(f,fig_obs,para, plot_para);

In [ ]:
on(pause_button.clicks) do shit
    i = 0
    @async while fig_obs.Isrunning[]
    # for unknown reasons loop does not work without @async,only use line below for debugging 
    #while fig_obs.Isrunning[]
        isopen(f.scene) || break 
        para = para_obs[]
        if fig_obs.Isrunning[] == false
            sleep(0.01)
            continue
        else
            diffusion_2D!(grids,diff_grids,para)
            i += 1

            if (i%para.pa_ph.ratio_walker_diff) == 0 
                update_directions!(grids, Agent_list_cicular, para)
                update_position!(grids, Agent_list_cicular, para)
                strenghten_boundary!(grids, para)
                divison!(grids, Agent_list_cicular,para)
                adsorb!(grids, Agent_list_cicular, para)
                animation_step!(ax1,ax2,Agent_list_cicular,grids,fig_obs, plot_para,para)
            end
        end
    end
end

ObserverFunction defined at In[6]:2 operating on Observable{Any}(0)

button clicked
false
button clicked
true


In [15]:
fig_obs.Time[] = 1
record(f, "vids/TrypColonies_no_arrows.mp4",  2:para.timesteps, framerate = plot_para.framerate) do i
    for t in 1:para.pa_ph.ratio_walker_diff
        diffusion_2D!(grids,diff_grids,para)
    end
    
    update_directions!(grids, Agent_list_cicular, para)
    update_position!(grids, Agent_list_cicular, para)
    strenghten_boundary!(grids, para)
    divison!(grids, Agent_list_cicular,para)
    adsorb!(grids, Agent_list_cicular, para)
    animation_step!(ax1,ax2,Agent_list_cicular,grids,fig_obs, plot_para,para)
end

"vids/TrypColonies_no_arrows.mp4"

In [12]:
@h Makie.update!

```julia
update!(graph; kwargs...)
update!(graph, pairs::Pair{Symbol, Any}...)
```

Updates any number of inputs in the graph based on the passed `key = value` keyword arguments. The `key` refers to the name of the input and the `value` is the new value.

## Example:

```julia
graph = ComputeGraph()
add_input!(graph, :first_node, 1)
update!(graph, first_node = 2)
update!(graph, :first_node => 2)
```


In [ ]:
t = 1
for i = 1:(para.pa_ph.time_steps_to_compute-1)

    diffusion_2D!(grids,diff_grids,para)
    t += 1

    if (i%para.pa_ph.ratio_walker_diff) == 0 
        update_directions!(grids, Agent_list_cicular, para)
        update_position!(grids, Agent_list_cicular, para)
        strenghten_boundary!(grids, para)
        divison!(grids, Agent_list_cicular,para)
        adsorb!(grids, Agent_list_cicular, para)
        animation_step!(ax1,Agent_list_cicular,grids,fig_obs, plot_para,para, toggle)

        dist_vec, dist_vec2 =  TrypColonies.radial_distribution( fig_obs.X[], fig_obs.Y[], para.size)
    end

end


In [ ]:
fig_obs.Time[] = 1
record(f, "vids/TrypColonies_test.mp4",  2:para.timesteps, framerate = plot_para.framerate) do i
    para = para_obs[]
    update_directions!(grids, Agent_list_cicular, para)
    #update_directions_diff!(grid, Agent_list_cicular, para, scal_grid)

    update_position!(grids, Agent_list_cicular, para)

    strenghten_boundary!(grids, para)
    divison!(grids, Agent_list_cicular,para)

    animation_step!(ax1,Agent_list_cicular,grids,fig_obs, plot_para,para)
end

"vids/TrypColonies_test.mp4"

# Basic Model in confinement 

In [30]:
### parameter for simulation in periodic boundary conditions    

para = parameters(
    timesteps           = 1000,
    agent_number        = 2000,
    size                = (500,500),  ### very strange GLMakie error, when size has certain big values like 5000 heatmap bugs out but e.g. 5004 works fine ? no idea why  
    velocity            = 2,
    interaction_radius  = 6,
    noise_dis           = Normal,
    noise_strength      = 0.1,
    boundary_conditions = "periodic",
    velocity_dis        = Normal, 
    velocity_variance   = 1.0,
    geometry           = "circle_small",
    start_config        = "random",
    radius_tanget       = 10,
    boundary_moveable   = true,
    radius_collision    = 3,
    path_tracing        = true, 
    sliding_boundary    = true, 
    grid_strength       = 10,
    grid_recover_rate   = 0.0,
    repulsion_flag      = true,
    repulsion_range     = 0,
    chemotaxis_flag     = false,
    growth_rate         = 0.005,
    )

    plot_para = plot_parameters(
    arrow_tip_size   = 8,
    arrow_tail_length= 5,
    framerate        = 60, 
    theme            = theme_black(),
    fontsize         = 15,
    refresh_delay    = 0.005,
    res              = (900,1300),
    cor_func         = order_parameter
    );

In [ ]:

grid = create_grid(para)
grid, Full_agent_list = initialize_system(grid,para)
Grid, Agent_list_cicular = make_sys_interactive!(grid,Full_agent_list)



makie_config!(plot_para)

f = Figure(size = plot_para.res)
display(f)
ax1,fig_obs = ini_animation(Agent_list_cicular,para,Grid,f,
    plot_para);

pause_button,para_obs = create_buttons(f,fig_obs,para);

In [32]:
on(pause_button.clicks) do shit
    @async while fig_obs.Isrunning[]
    # for unknown reasons loop does not work without @async,only use line below for debugging 
    #while fig_obs.Isrunning[]
        #break if scene is closed or stop button clicked
        isopen(f.scene) || break 
        para = para_obs[]
        if fig_obs.Isrunning[] == false
            sleep(0.01)
            continue
        else
            update_directions!(grid, Agent_list_cicular, para)
            update_position!(grid, Agent_list_cicular, para)
            strenghten_boundary!(grid, para)
            divison!(grid, Agent_list_cicular,para)
            animation_step!(ax1,Agent_list_cicular,grid,fig_obs, plot_para,para)
        end
    end
end

ObserverFunction defined at c:\Users\ank10ki\Dropbox\phd\TrypColonies\TrypColonies\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:2 operating on Observable{Any}(0)

# Chemotaxis 

In [8]:
### parameter for simulation in periodic boundary conditions    

para = parameters(
    timesteps           = 1000,
    agent_number        = 2000,
    size                = (500,500),  ### very strange GLMakie error, when size has certain big values like 5000 heatmap bugs out but e.g. 5004 works fine ? no idea why  
    velocity            = 2,
    interaction_radius  = 1,
    noise_dis           = Normal,
    noise_strength      = 0.3,
    boundary_conditions = "periodic",
    velocity_dis        = Normal, 
    velocity_variance   = 1.0,
    geometry           = "circle_small",
    start_config        = "random",
    radius_tanget       = 10,
    boundary_moveable   = true,
    radius_collision    = 3,
    path_tracing        = true, 
    sliding_boundary    = true, 
    grid_strength       = 10,
    grid_recover_rate   = 0.0,
    repulsion_flag      = true,
    repulsion_range     = 0,
    growth_rate         = 0.005,
    )

    plot_para = plot_parameters(
    arrow_tip_size   = 4,
    arrow_tail_length= 3,
    framerate        = 60, 
    theme            = theme_black(),
    fontsize         = 15,
    refresh_delay    = 0.005,
    res              = (900,1300),
    cor_func         = order_parameter
    );

In [ ]:

grid = create_grid(para)
grid, Full_agent_list = initialize_system(grid,para)
Grid, Agent_list_cicular = make_sys_interactive!(grid,Full_agent_list)

scal_grid = create_exp_scalar_grid(para)


makie_config!(plot_para)

f = Figure(size = plot_para.res)
display(f)
ax1,fig_obs = ini_animation(Agent_list_cicular,para,Grid,scal_grid,f,
    plot_para);

pause_button,para_obs = create_buttons(f,fig_obs,para);

In [10]:
on(pause_button.clicks) do shit
    @async while fig_obs.Isrunning[]
    # for unknown reasons loop does not work without @async,only use line below for debugging 
    #while fig_obs.Isrunning[]
        #break if scene is closed or stop button clicked
        isopen(f.scene) || break 
        para = para_obs[]
        if fig_obs.Isrunning[] == false
            sleep(0.01)
            continue
        else
            update_directions!(grid, Agent_list_cicular, para, scal_grid)
            update_position!(grid, Agent_list_cicular, para)
            strenghten_boundary!(grid, para)
            divison!(grid, Agent_list_cicular,para)
            animation_step!(ax1,Agent_list_cicular,grid,fig_obs, plot_para,para)
        end
    end
end

ObserverFunction defined at c:\Users\Andreas\Dropbox\phd\TrypColonies\TrypColonies\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:2 operating on Observable{Any}(0)

In [15]:
fig_obs.Time[] = 1
record(f, "vids/TrypColonies_first.mp4",  2:para.timesteps, framerate = plot_para.framerate) do i
    para = para_obs[]
    update_directions!(grid, Agent_list_cicular, para)
    #update_directions_diff!(grid, Agent_list_cicular, para, scal_grid)

    update_position!(grid, Agent_list_cicular, para)

    strenghten_boundary!(grid, para)

    animation_step!(ax1,Agent_list_cicular,grid,fig_obs, plot_para,para)
end

BoundsError: BoundsError: attempt to access 2×2 Matrix{Float64} at index [306, 660]